<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LLM_for_GradioApps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 LangChain と OpenRouter によるチャットアプリケーション開発

## 📝 1. はじめに

このノートブックでは、 **LangChain** フレームワークを使用し、 **OpenRouter** 経由で大規模言語モデル（LLM）と対話する方法を学びます。

### チャット形式

言語モデルは根本的には「統計的に続きの文章を書く」という単純な仕組みですが、現在のモデルは人間との対話を目的として調整（チューニング）されています。そのため、単純な補完よりも「問いかけに対して適切な回答を返す」という対話形式で用いることが一般的です。

### OpenRouter
LLM を動かすには膨大な計算資源（ **GPU** 等）が必要ですが、個人で用意するのは困難です。 **OpenRouter** を利用することで、世界中の高性能なモデルを API 経由で手軽に利用できるようになります。

In [ ]:
# 必要なライブラリのインストール
!pip install -q langchain-openai termcolor tqdm

from termcolor import colored
print(colored("ライブラリの準備が完了しました。", "green"))

## 🔑 2. API キーの設定

Google Colab のシークレット機能（左側の鍵アイコン）に `OPENROUTER_API_KEY` を登録し、このノートブックからのアクセスを許可してください。

In [ ]:
from google.colab import userdata

# キーの存在確認のみ行います
if userdata.get("OPENROUTER_API_KEY"):
    print("API キーが正しく設定されています。")

### 🔑 2. API キーの設定（演習）

API キーなどの機密情報は、ソースコードに直接記述せず、 **環境変数** や Google Colab の **シークレット機能** を利用して管理します。これにより、ノートブックを共有した際の流出リスクを低減できます。

In [ ]:
from google.colab import userdata
from termcolor import colored

# 演習: 穴埋め箇所を埋めて、シークレットからキーを取得してください
api_key = ■■■.■■■("■■■")

if api_key:
    print(colored("API キーの取得に成功しました。", "green"))
else:
    print(colored("API キーが設定されていません。左側の鍵アイコンから設定してください。", "red"))

## 🧠 3. Chat Completion による対話の実装

LangChainの `ChatOpenAI` クラスを使用します。これは、メッセージの役割を定義して対話を行うためのコンポーネントです。

### メッセージの種類
* **SystemMessage**: AIの振る舞いや前提条件を指定します。
* **HumanMessage**: ユーザーからの具体的な問いかけです。

### 実行メソッド
* **invoke**: 単一の応答をまとめて取得します。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from google.colab import userdata

# クライアントの初期化
# base_url を OpenRouter のエンドポイントに変更します
chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="deepseek/deepseek-chat-v3-0324"
)

# メッセージリストの作成
messages = [
    SystemMessage(content="あなたは大学生向けのプログラミング講師です。"),
    HumanMessage(content="プログラミングを学ぶとすれば、まず何から手をつけたらいいですか？ 100文字以内で新しい入学者に向けて答えて")
]

# まとめて結果を取得
response = chat.invoke(messages)

print(f"回答内容:\n{response.content}")

### 🧠 3. Chat Completion による対話の実装（演習）

LangChain を介して LLM と対話する際は、メッセージの役割を明確に定義します。

* **SystemMessage**: モデルの役割や制約を定義します。
* **HumanMessage**: ユーザーからの入力を定義します。
* **AIMessage**: モデルからの返答を表します。

また、実行方法には一括で結果を受け取る **invoke** と、生成過程を逐次取得する **stream** があります。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from google.colab import userdata

# 演習: ChatOpenAI クライアントを初期化してください
# 非推論モデル (deepseek/deepseek-chat-v3-0324) を使用します
chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="■■■"
)

# メッセージリストの作成
messages = [
    ■■■(content="あなたは数学の家庭教師です。"),
    ■■■(content="二次方程式の解の公式を簡潔に教えてください。")
]

# 実行（一括取得）
response = chat.■■■(messages)

print(f"回答:\n{response.content}")

### 実行メソッド
* **stream**: 生成されたテキストを、分割されたデータ（チャンク）としてリアルタイムに取得します。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from google.colab import userdata

# 思考プロセスを伴うモデルに対して、逐次出力を適用します
stream_chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="deepseek/deepseek-chat-v3-0324"
)

question = "LangChain を使う最大のメリットを 3 つ挙げてください。"
messages = [HumanMessage(content=question)]

print("ストリーミング出力を開始します:\n")

# chunk ごとに処理を行います
for chunk in stream_chat.stream(messages):
    # content を改行なしで表示します
    print(chunk.content, end="", flush=True)

print("\n\n出力が完了しました。")

ここでは演習は用意していません。

## ⛓️ 4. LCEL による処理の連鎖（Chain）

LangChain の **LCEL** (LangChain Expression Language) を使用すると、複数のコンポーネントを **パイプ記号** ( `|` ) で連結し、データの流れを直感的に記述できます。

### 基本構成
* **PromptTemplate**: ユーザー入力を受け取り、 LLM に渡すための定型文を作成します。
* **StrOutputParser**: LLM からの応答（ChatMessage オブジェクト）から、テキスト部分のみを抽出します。

これらを組み合わせることで、入力から出力までを一貫した **Chain** として定義できます。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

# モデルの初期化
chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="deepseek/deepseek-chat-v3-0324"
)

# プロンプトテンプレートの定義
prompt = ChatPromptTemplate.from_template("{topic} について、専門家として簡潔に解説してください。")

# 出力パーサーの定義
output_parser = StrOutputParser()

# パイプ記号による連結
chain = prompt | chat | output_parser

# 実行
topic_input = "量子コンピュータ"
result = chain.invoke({"topic": topic_input})

print(f"トピック: {topic_input}")
print(f"結果:\n{result}")

### 📝 演習 4: Chain の構築
以下のコードを完成させ、入力された料理のレシピを考案する Chain を作成してください。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="deepseek/deepseek-chat-v3-0324"
)

# 演習: 穴埋め箇所を埋めてください
prompt = ChatPromptTemplate.from_template("■■■ を使った簡単なレシピを教えてください。")
output_parser = ■■■()

# Chain の連結
recipe_chain = ■■■ | ■■■ | ■■■

# 実行
ingredients = "卵とトマト"
print(recipe_chain.invoke({"topic": ingredients}))

## 🤖 5. Gradio によるチャットインターフェースの構築

### Gradio とは
**Gradio** は、 Python コードのみで機械学習モデルや任意の関数のウェブインターフェース（ GUI ）を構築するためのライブラリです。 HTML や CSS 、 JavaScript の知識がなくても、ブラウザ上で操作可能なアプリケーションを作成できます。

### GUI を設計する意義
プログラムをコンソール（コマンドライン）だけで実行する場合、入力形式が固定され、一般の利用者が操作するには難易度が高くなります。 **GUI** （ Graphical User Interface ）を設計することで、テキストボックス、ボタン、スライダといった視覚的な部品を配置し、直感的な操作が可能になります。

### ChatInterface の役割
Gradio が提供する `gr.ChatInterface` クラスは、チャットボットに特化した高レベルなコンポーネントです。以下の機能が標準で備わっています。

* ユーザーの入力用テキストボックス
* 送信（ Submit ）ボタンおよび停止（ Stop ）ボタン
* チャット履歴を表示するウィンドウ
* 過去の対話内容の管理

このインターフェースでは、ユーザーが送信ボタンを押した際、指定した関数（ `predict` など）が自動的に呼び出されます。

In [ ]:
# Gradio のインストール
!pip install -q gradio

import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from google.colab import userdata

chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="deepseek/deepseek-chat-v3-0324"
)

def chat_response(message, history):
    # システムプロンプトで役割を固定
    messages = [SystemMessage(content="あなたはカスタマーセンターのオペレータです。鉄道の運行状況についての案内を日本語で行ってください。")]

    for h_msg, a_msg in history:
        messages.append(HumanMessage(content=h_msg))
        messages.append(AIMessage(content=a_msg))

    messages.append(HumanMessage(content=message))

    response = ""
    for chunk in chat.stream(messages):
        response += chunk.content
        yield response

# チャットインターフェースの起動
gr.ChatInterface(fn=chat_response).launch(share=True)

## 🛠️ 6. Gradio Blocks による詳細な UI 設計

### Blocks による柔軟なレイアウト
`gr.ChatInterface` は便利ですが、画面の構成を自由に変更することはできません。より複雑なアプリケーションを作るには、 **gr.Blocks** を使用します。これは、コンポーネントをブロックのように組み合わせて配置するものです。

### GUI コンポーネントの種類
* **gr.Textbox**: テキストの入出力に使用します。 `lines` 引数で高さを調節できます。
* **gr.Markdown**: 文書やタイトルを整形して表示します。
* **gr.Row** / **gr.Column**: コンポーネントを横並び、あるいは縦並びに配置するためのコンテナです。
* **gr.Button**: 処理の実行トリガーとなります。

### イベント処理の仕組み（ click 時の動作 ）
Blocks では、どのボタンが押されたときに、どの関数を実行するかを明示的に記述する必要があります。

```python
submit_btn.click(fn=my_function, inputs=input_component, outputs=output_component)
```

この引数を見てみましょう。
- fn: ボタンがクリックされたときに実行される関数を指定します。
- inputs: 関数に引数として渡すコンポーネントを指定します。
- outputs: 関数の戻り値を表示するコンポーネントを指定します。

この仕組みにより、特定の入力フィールドの値を読み取り、計算や LLM による処理を行った結果を、特定の出力フィールドに書き戻すという一連の流れを実装できます。

In [ ]:
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="deepseek/deepseek-chat-v3-0324"
)

def summarize_text(text):
    prompt = ChatPromptTemplate.from_template("以下の文章を3行の箇条書きで要約してください。\n\n{input}")
    chain = prompt | chat | StrOutputParser()
    return chain.invoke({"input": text})

# カスタムレイアウトの定義
with gr.Blocks() as demo:
    gr.Markdown("# 📝 文章要約ツール")

    with gr.Row():
        input_area = gr.Textbox(label="元文章", lines=10, placeholder="ここに要約したい文章を入力してください...")
        output_area = gr.Textbox(label="要約結果", lines=5)

    submit_btn = gr.Button("要約を実行")

    # ボタンクリック時の動作を定義
    submit_btn.click(fn=summarize_text, inputs=input_area, outputs=output_area)

demo.launch()

### 📝 演習 6: カスタム UI の構築
以下のコードを完成させ、入力された単語からキャッチコピーを生成するツールを作成してください。

In [ ]:
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

chat = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY"),
    model="deepseek/deepseek-chat-v3-0324"
)

def generate_copy(product_name):

  # プロンプトテンプレート。{product_name} には、あとでこの関数の引数が入る
    prompt = ■■■.■■■("製品名「{product_name}」の魅力的なキャッチコピーを 1 つ提案してください。その1行だけ出力。")

  # チェーン。プロンプトをチャットに与えて、その出力を解釈するという一連の流れ
    chain = ■■■ | ■■■ | ■■■()

  # product_name を指定して、上記のチェーンを実行
    return ■■■.invoke({"product_name": product_name})

# UI コンポーネントの配置
with gr.Blocks() as copy_tool:
    gr.Markdown("## 💡 キャッチコピー生成機")

    # 入力も出力もテキストボックスで行う
    word_input = gr.■■■(label="製品名・キーワード")
    copy_output = gr.■■■(label="提案内容")

    # 生成開始ボタン
    run_button = gr.■■■("生成開始")

    # 処理の紐付け
    run_button.click(fn=■■■, inputs=word_input, outputs=■■■)

# 実行
copy_tool.launch()